In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
tagged_sentences=nltk.corpus.treebank.tagged_sents()
import os
print(os.listdir("../input"))
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
# Any results you write to the current directory are saved as output.

[]


Using TensorFlow backend.


In [2]:
len(tagged_sentences)

3914

In [3]:
N=len(tagged_sentences)

In [4]:
N

3914

In [5]:
taggedwords=nltk.corpus.treebank.tagged_words()

In [6]:
taggedwords[0]  # all words being used with their tags

('Pierre', 'NNP')

In [7]:
sentences, sentence_tags =[], [] 
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))

In [8]:
sentences[0]

array(['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join',
       'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.',
       '29', '.'], dtype='<U12')

In [9]:
len(sentences[8])==len(sentence_tags[8])

True

In [10]:
trainset,testset,ytrain,ytest=train_test_split(sentences,sentence_tags,test_size=0.2)

In [11]:
# Preprocessing with data
len(testset)

783

In [12]:
trainset[0]
# arrays can stores strings
# before applying model tokenize each training example text into word tokens
# training set should have list of word tokens for each training example

array(['*-1', 'Moving', 'rapidly', 'through', 'school', ',', 'he',
       'graduated', 'Phi', 'Beta', 'Kappa', 'from', 'the', 'University',
       'of', 'Kentucky', 'at', 'age', '18', ',', 'after', '*', 'spending',
       'only', '2', '1\\/2', 'years', 'in', 'college', '.'], dtype='<U10')

In [13]:
# Since keras works only with numbers so we need to convert each word by unique index and with tags too

In [14]:
# VERY IMPORTANT PREPROCESSING STEP #
words,tags=set([]),set([])

for s in trainset:
    for w in s:
        words.add(w.lower())
        
for ts in ytrain:
    for t in ts:
        tags.add(t)


In [15]:
word2index={j:i+2 for i,j in enumerate(list(words))}
word2index['-PAD-']=0    # Pad  sequences in starting or ending
word2index['-OOV-']=1    # if unknown word in sentences replace it with OOV

In [16]:
tag2index={t:i+1 for i,t in enumerate(list(tags))}
tag2index['-PAD-']=0

In [17]:
train_sent_x,test_sent_x,train_tags_y,test_tags_y=[],[],[],[]

In [18]:
for s in trainset:
    m=[]
    for i in s:
        try:
            m.append(word2index[i.lower()])
        except:
            m.append(word2index['-OOV-'])
    train_sent_x.append(m)

In [19]:
for s in testset:
    m=[]
    for i in s:
        try:
            m.append(word2index[i.lower()])
        except:
            m.append(word2index['-OOV-'])
    test_sent_x.append(m)

In [20]:
len(test_sent_x)    #remember 1 stands for unknown word tag

783

In [21]:
# Here are assuming that virtually all tags are present in our training dataset
# which will be present in test data set also
# there is no tag that is present in test dataset but not in training dataset
# VERY IMPORTANT PART OF BUILDING Algorithm


In [22]:
# Replacing Tags with indexes in train labels
for s in ytrain:
    m=[]
    for j in s:
        m.append(tag2index[j])
    train_tags_y.append(m)

In [23]:
len(train_tags_y[0])

30

In [24]:
len(ytrain[0])
len(ytest)

783

In [25]:
#Replacing Tags with indexes in test labels
for s in ytest:
    m=[]
    for j in s:
        m.append(tag2index[j])
    test_tags_y.append(m)

In [26]:
len(ytest)
len(test_tags_y)

783

In [27]:
print(train_sent_x[0])
print(trainset[0])
print(ytrain[0])
print(train_tags_y[0])

[589, 1395, 8982, 1761, 7837, 1119, 2660, 4729, 4671, 6095, 7694, 2320, 42, 10108, 5740, 4758, 2006, 5908, 4414, 1119, 1195, 8427, 24, 3648, 1807, 1023, 4510, 7716, 3289, 1585]
['*-1' 'Moving' 'rapidly' 'through' 'school' ',' 'he' 'graduated' 'Phi'
 'Beta' 'Kappa' 'from' 'the' 'University' 'of' 'Kentucky' 'at' 'age' '18'
 ',' 'after' '*' 'spending' 'only' '2' '1\\/2' 'years' 'in' 'college' '.']
['-NONE-' 'VBG' 'RB' 'IN' 'NN' ',' 'PRP' 'VBD' 'NNP' 'NNP' 'NNP' 'IN' 'DT'
 'NNP' 'IN' 'NNP' 'IN' 'NN' 'CD' ',' 'IN' '-NONE-' 'VBG' 'RB' 'CD' 'CD'
 'NNS' 'IN' 'NN' '.']
[27, 17, 31, 19, 7, 2, 41, 23, 24, 24, 24, 19, 40, 24, 19, 24, 19, 7, 16, 2, 19, 27, 17, 31, 16, 16, 33, 19, 7, 18]


In [28]:
#we have make size of input sequences to be fed into keras same as
# that is how text data is put into Embedding

In [29]:
maxlen=len(max(train_sent_x,key=len))

In [30]:
maxlen

271

In [31]:
train_sent_x[1]

[209,
 1924,
 6653,
 27,
 589,
 1431,
 42,
 9297,
 5740,
 4143,
 5261,
 4583,
 8086,
 6922,
 9405,
 6695,
 6538,
 6958,
 8101,
 1585]

In [32]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense,Dropout,LSTM,InputLayer,Bidirectional,TimeDistributed,Embedding,Activation

In [33]:
train_sequences_X=pad_sequences(train_sent_x,maxlen=maxlen,padding='post')
test_sequences_x=pad_sequences(test_sent_x,maxlen=maxlen,padding='post')
train_tagsy=pad_sequences(train_tags_y,maxlen=maxlen,padding='post')
test_tagsy=pad_sequences(test_tags_y,maxlen=maxlen,padding='post')

In [34]:
len(test_tags_y)
len(test_tagsy)

783

In [35]:
test_tagsy[0]


array([24, 24,  2, 16, 33, 12,  2, 46, 10, 40,  7, 19, 40, 12,  7, 24, 16,
       18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [36]:
train_tagsy[0]

array([27, 17, 31, 19,  7,  2, 41, 23, 24, 24, 24, 19, 40, 24, 19, 24, 19,
        7, 16,  2, 19, 27, 17, 31, 16, 16, 33, 19,  7, 18,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [37]:
tag2index['-PAD-']

0

In [38]:
model=Sequential()
model.add(InputLayer(input_shape=(maxlen,)))
model.add(Embedding(len(word2index),128))
model.add(Bidirectional(LSTM(256,return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 271, 128)          1300608   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 271, 512)          788480    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 271, 47)           24111     
_________________________________________________________________
activation_1 (Activation)    (None, 271, 47)           0         
Total params: 2,113,199
Trainable params: 2,113,199
Non-trainable params: 0
_________________________________________________________________


In [40]:
#remember number of timestamps is always at middle of dims
train_tags_y[0]

[27,
 17,
 31,
 19,
 7,
 2,
 41,
 23,
 24,
 24,
 24,
 19,
 40,
 24,
 19,
 24,
 19,
 7,
 16,
 2,
 19,
 27,
 17,
 31,
 16,
 16,
 33,
 19,
 7,
 18]

In [41]:
train_tags_encode_y=to_categorical(train_tagsy,len(tag2index))
test_tags_encode_y=to_categorical(test_tagsy,len(tag2index))

In [42]:
print(train_tagsy.shape)
print(train_tags_encode_y.shape)

(3131, 271)
(3131, 271, 47)


In [43]:
print(test_tagsy.shape)

(783, 271)


In [44]:
len(train_tags_encode_y)
len(test_tags_encode_y)

783

In [45]:
model.compile(optimizer=Adam(0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [46]:
history=model.fit(train_sequences_X,train_tags_encode_y,batch_size=128,epochs=40,validation_split=0.2,verbose=1)

Train on 2504 samples, validate on 627 samples
Epoch 1/40
2504/2504 [==============================] - 20s 8ms/step - loss: 1.4452 - acc: 0.8583 - val_loss: 0.4136 - val_acc: 0.9035
Epoch 2/40
2504/2504 [==============================] - 17s 7ms/step - loss: 0.3470 - acc: 0.9064 - val_loss: 0.3216 - val_acc: 0.9046
Epoch 3/40
2504/2504 [==============================] - 18s 7ms/step - loss: 0.3155 - acc: 0.9103 - val_loss: 0.3101 - val_acc: 0.9166
Epoch 4/40
2504/2504 [==============================] - 19s 7ms/step - loss: 0.3057 - acc: 0.9166 - val_loss: 0.3020 - val_acc: 0.9168
Epoch 5/40
2504/2504 [==============================] - 17s 7ms/step - loss: 0.2976 - acc: 0.9167 - val_loss: 0.2944 - val_acc: 0.9169
Epoch 6/40
2504/2504 [==============================] - 17s 7ms/step - loss: 0.2898 - acc: 0.9169 - val_loss: 0.2868 - val_acc: 0.9179
Epoch 7/40
2504/2504 [==============================] - 17s 7ms/step - loss: 0.2829 - acc: 0.9183 - val_loss: 0.2809 - val_acc: 0.9206
Epoch 8/

In [47]:
print(test_tags_encode_y.shape)
print(train_sequences_X.shape)
print(test_sequences_x.shape)

(783, 271, 47)
(3131, 271)
(783, 271)


In [48]:
scores=model.evaluate(test_sequences_x,test_tags_encode_y)

783/783 [==============================] - 7s 9ms/step


In [49]:
scores

[0.03239622550373029, 0.9918329069440849]

In [50]:
ytest[0]

array(['NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'MD', 'VB', 'DT', 'NN',
       'IN', 'DT', 'JJ', 'NN', 'NNP', 'CD', '.'], dtype='<U3')

In [51]:
testset[0]

array(['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join',
       'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.',
       '29', '.'], dtype='<U12')